In [2]:
import os
import chromadb
from chromadb.utils import embedding_functions
from pypdf import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid
import time

Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 6
Python-dotenv could not parse statement starting at line 6


In [3]:
# --- Configuration ---
PDF_FOLDER = "/Users/devanshk/Desktop/Deepseek-r1-local-chatbot/rag-data"
CHROMA_DB_PATH = "./chroma_db_persistent"  # Directory to store persistent ChromaDB data
COLLECTION_NAME = "pdf_semantic_chunks"
EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"  # Make sure this model is compatible with SentenceTransformerEmbeddingFunction

In [4]:
!pip install sentence_transformers
# --- Initialize ChromaDB Client (Persistent) ---
print(f"Initializing ChromaDB client at: {CHROMA_DB_PATH}")
# Creates the directory if it doesn't exist and persists data there
client = chromadb.PersistentClient(path=CHROMA_DB_PATH)

# --- Initialize Embedding Function ---
print(f"Using embedding model: {EMBEDDING_MODEL_NAME}")
# Using a helper function from ChromaDB Utils for Sentence Transformers
# This handles downloading the model and generating embeddings automatically within ChromaDB
chroma_embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name=EMBEDDING_MODEL_NAME
)

Python-dotenv could not parse statement starting at line 6


Initializing ChromaDB client at: ./chroma_db_persistent
Using embedding model: all-MiniLM-L6-v2


In [5]:
# --- Get or Create ChromaDB Collection ---
print(f"Getting or creating ChromaDB collection: {COLLECTION_NAME}")
# Pass the embedding function during collection creation/retrieval
# ChromaDB will use this function automatically when you add documents if embeddings aren't provided
collection = client.get_or_create_collection(
    name=COLLECTION_NAME,
    embedding_function=chroma_embedding_function,
    metadata={"hnsw:space": "cosine"}  # Optional: Specifies the distance metric (cosine is common for text embeddings)
)


Getting or creating ChromaDB collection: pdf_semantic_chunks


In [6]:
# --- Initialize Text Splitter (using LangChain for structure-aware chunking) ---
# This splitter tries to keep paragraphs/sentences together, which is a form of semantic chunking.
# Adjust chunk_size and chunk_overlap as needed for your specific documents and use case.
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # Max characters per chunk
    chunk_overlap=150,       # Characters to overlap between chunks
    separators=["\n\n", "\n", ". ", "? ", "! ", " ", ""], # Order of separators to try
    length_function=len,     # Function to measure chunk size (usually len)
)

In [7]:
# --- Process PDF Files ---
print(f"\nStarting PDF processing from folder: {PDF_FOLDER}")
processed_files = 0
total_chunks_added = 0

# Check if the folder exists
if not os.path.isdir(PDF_FOLDER):
    print(f"Error: Folder '{PDF_FOLDER}' not found.")
    exit()

pdf_files = [f for f in os.listdir(PDF_FOLDER) if f.lower().endswith(".pdf")]

if not pdf_files:
    print(f"No PDF files found in '{PDF_FOLDER}'.")
    exit()

print(f"Found {len(pdf_files)} PDF files to process.")

start_time = time.time()

for pdf_file in pdf_files:
    pdf_path = os.path.join(PDF_FOLDER, pdf_file)
    print(f"\nProcessing: {pdf_path}...")

    try:
        # 1. Read PDF
        reader = PdfReader(pdf_path)
        full_text = ""
        for i, page in enumerate(reader.pages):
            page_text = page.extract_text()
            if page_text:
                full_text += f"\n--- Page {i+1} ---\n" + page_text # Add page breaks for context
            else:
                 print(f" - Warning: Could not extract text from page {i+1} of {pdf_file}")


        if not full_text.strip():
            print(f" - Warning: No text extracted from {pdf_file}. Skipping.")
            continue

        # 2. Perform Semantic Chunking (Structure-Aware Splitting)
        chunks = text_splitter.split_text(full_text)

        if not chunks:
            print(f" - Warning: Text splitting resulted in no chunks for {pdf_file}. Skipping.")
            continue

        print(f" - Extracted text and split into {len(chunks)} chunks.")

        # 3. Prepare Data for ChromaDB
        ids = [f"{pdf_file}_chunk_{i}_{uuid.uuid4()}" for i in range(len(chunks))] # Unique IDs are crucial
        documents = chunks # The actual text content of each chunk
        metadatas = [{"source_file": pdf_file, "original_chunk_index": i} for i in range(len(chunks))] # Metadata per chunk

        # 4. Add to ChromaDB Collection
        # ChromaDB will automatically use the collection's embedding function
        # to generate embeddings for the 'documents' being added.
        collection.add(
            documents=documents,
            metadatas=metadatas,
            ids=ids
        )
        print(f" - Added {len(chunks)} chunks to ChromaDB collection '{COLLECTION_NAME}'.")
        processed_files += 1
        total_chunks_added += len(chunks)

    except Exception as e:
        print(f" - !!! Error processing {pdf_file}: {e}")
        # Optionally add more robust error handling (e.g., logging to a file)

# --- Final Summary ---
end_time = time.time()
elapsed_time = end_time - start_time

print("\n--- Processing Complete ---")
print(f"Successfully processed {processed_files} out of {len(pdf_files)} PDF files.")
print(f"Total chunks added to ChromaDB: {total_chunks_added}")
print(f"Total time taken: {elapsed_time:.2f} seconds")
try:
    print(f"ChromaDB collection '{COLLECTION_NAME}' now contains {collection.count()} items.")
    print(f"Persistent data stored in: {os.path.abspath(CHROMA_DB_PATH)}")
except Exception as e:
    print(f"Could not get final count from ChromaDB collection: {e}")


Starting PDF processing from folder: /Users/devanshk/Desktop/Deepseek-r1-local-chatbot/rag-data
Found 3 PDF files to process.

Processing: /Users/devanshk/Desktop/Deepseek-r1-local-chatbot/rag-data/Data Document.pdf...
 - Extracted text and split into 90 chunks.


Ignoring wrong pointing object 0 0 (offset 0)


 - Added 90 chunks to ChromaDB collection 'pdf_semantic_chunks'.

Processing: /Users/devanshk/Desktop/Deepseek-r1-local-chatbot/rag-data/The Now Habit.pdf...
 - Extracted text and split into 19 chunks.
 - Added 19 chunks to ChromaDB collection 'pdf_semantic_chunks'.

Processing: /Users/devanshk/Desktop/Deepseek-r1-local-chatbot/rag-data/Deep Work by Cal Newport.pdf...
 - Extracted text and split into 21 chunks.
 - Added 21 chunks to ChromaDB collection 'pdf_semantic_chunks'.

--- Processing Complete ---
Successfully processed 3 out of 3 PDF files.
Total chunks added to ChromaDB: 130
Total time taken: 2.94 seconds
ChromaDB collection 'pdf_semantic_chunks' now contains 130 items.
Persistent data stored in: /Users/devanshk/Downloads/Deepseek-r1-local-chatbot/chroma_db_persistent
